In [3]:
'''
Reproducing paper:
Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun. "Identity Mappings in Deep Residual Networks"
'''
import mxnet as mx

def residual_unit(data, num_filter, stride, dim_match, name, bottle_neck=True, bn_mom=0.9, workspace=512, memonger=False):
    """Return ResNet Unit symbol for building ResNet
    Parameters
    ----------
    data : str
        Input data
    num_filter : int
        Number of output channels
    bnf : int
        Bottle neck channels factor with regard to num_filter
    stride : tupe
        Stride used in convolution
    dim_match : Boolen
        True means channel number between input and output is the same, otherwise means differ
    name : str
        Base name of the operators
    workspace : int
        Workspace used in convolution operator
    """
    if bottle_neck:
        # the same as https://github.com/facebook/fb.resnet.torch#notes, a bit difference with origin paper
        bn1 = mx.sym.BatchNorm(data=data, fix_gamma=False, eps=2e-5, momentum=bn_mom, name=name + '_bn1')
        act1 = mx.sym.Activation(data=bn1, act_type='relu', name=name + '_relu1')
        conv1 = mx.sym.Convolution(data=act1, num_filter=int(num_filter*0.25), kernel=(1,1), stride=(1,1), pad=(0,0),
                                      no_bias=True, workspace=workspace, name=name + '_conv1')
        bn2 = mx.sym.BatchNorm(data=conv1, fix_gamma=False, eps=2e-5, momentum=bn_mom, name=name + '_bn2')
        act2 = mx.sym.Activation(data=bn2, act_type='relu', name=name + '_relu2')
        conv2 = mx.sym.Convolution(data=act2, num_filter=int(num_filter*0.25), kernel=(3,3), stride=stride, pad=(1,1),
                                      no_bias=True, workspace=workspace, name=name + '_conv2')
        bn3 = mx.sym.BatchNorm(data=conv2, fix_gamma=False, eps=2e-5, momentum=bn_mom, name=name + '_bn3')
        act3 = mx.sym.Activation(data=bn3, act_type='relu', name=name + '_relu3')
        conv3 = mx.sym.Convolution(data=act3, num_filter=num_filter, kernel=(1,1), stride=(1,1), pad=(0,0), no_bias=True,
                                   workspace=workspace, name=name + '_conv3')
        if dim_match:
            shortcut = data
        else:
            shortcut = mx.sym.Convolution(data=act1, num_filter=num_filter, kernel=(1,1), stride=stride, no_bias=True,
                                            workspace=workspace, name=name+'_sc')
        if memonger:
            shortcut._set_attr(mirror_stage='True')
        return conv3 + shortcut
    else:
        bn1 = mx.sym.BatchNorm(data=data, fix_gamma=False, momentum=bn_mom, eps=2e-5, name=name + '_bn1')
        act1 = mx.sym.Activation(data=bn1, act_type='relu', name=name + '_relu1')
        conv1 = mx.sym.Convolution(data=act1, num_filter=num_filter, kernel=(3,3), stride=stride, pad=(1,1),
                                      no_bias=True, workspace=workspace, name=name + '_conv1')
        bn2 = mx.sym.BatchNorm(data=conv1, fix_gamma=False, momentum=bn_mom, eps=2e-5, name=name + '_bn2')
        act2 = mx.sym.Activation(data=bn2, act_type='relu', name=name + '_relu2')
        conv2 = mx.sym.Convolution(data=act2, num_filter=num_filter, kernel=(3,3), stride=(1,1), pad=(1,1),
                                      no_bias=True, workspace=workspace, name=name + '_conv2')
        if dim_match:
            shortcut = data
        else:
            shortcut = mx.sym.Convolution(data=act1, num_filter=num_filter, kernel=(1,1), stride=stride, no_bias=True,
                                            workspace=workspace, name=name+'_sc')
        if memonger:
            shortcut._set_attr(mirror_stage='True')
        return conv2 + shortcut

def resnet(units, num_stage, filter_list, num_class, data_type, bottle_neck=True, bn_mom=0.9, workspace=512, memonger=False):
    """Return ResNet symbol of cifar10 and imagenet
    Parameters
    ----------
    units : list
        Number of units in each stage
    num_stage : int
        Number of stage
    filter_list : list
        Channel size of each stage
    num_class : int
        Ouput size of symbol
    dataset : str
        Dataset type, only cifar10 and imagenet supports
    workspace : int
        Workspace used in convolution operator
    """
    num_unit = len(units)
    assert(num_unit == num_stage)
    data = mx.sym.Variable(name='data')
    data = mx.sym.BatchNorm(data=data, fix_gamma=True, eps=2e-5, momentum=bn_mom, name='bn_data')
    if data_type == 'cifar10':
        body = mx.sym.Convolution(data=data, num_filter=filter_list[0], kernel=(3, 3), stride=(1,1), pad=(1, 1),
                                  no_bias=True, name="conv0", workspace=workspace)
    elif data_type == 'imagenet':
        body = mx.sym.Convolution(data=data, num_filter=filter_list[0], kernel=(7, 7), stride=(2,2), pad=(3, 3),
                                  no_bias=True, name="conv0", workspace=workspace)
        body = mx.sym.BatchNorm(data=body, fix_gamma=False, eps=2e-5, momentum=bn_mom, name='bn0')
        body = mx.sym.Activation(data=body, act_type='relu', name='relu0')
        body = mx.symbol.Pooling(data=body, kernel=(3, 3), stride=(2,2), pad=(1,1), pool_type='max')
    else:
         raise ValueError("do not support {} yet".format(data_type))
    for i in range(num_stage):
        body = residual_unit(body, filter_list[i+1], (1 if i==0 else 2, 1 if i==0 else 2), False,
                             name='stage%d_unit%d' % (i + 1, 1), bottle_neck=bottle_neck, workspace=workspace,
                             memonger=memonger)
        for j in range(units[i]-1):
            body = residual_unit(body, filter_list[i+1], (1,1), True, name='stage%d_unit%d' % (i + 1, j + 2),
                                 bottle_neck=bottle_neck, workspace=workspace, memonger=memonger)
    bn1 = mx.sym.BatchNorm(data=body, fix_gamma=False, eps=2e-5, momentum=bn_mom, name='bn1')
    relu1 = mx.sym.Activation(data=bn1, act_type='relu', name='relu1')
    # Although kernel is not used here when global_pool=True, we should put one
    pool1 = mx.symbol.Pooling(data=relu1, global_pool=True, kernel=(7, 7), pool_type='avg', name='pool1')
    flat = mx.symbol.Flatten(data=pool1)
    fc1 = mx.symbol.FullyConnected(data=flat, num_hidden=num_class, name='fc1')
    return mx.symbol.SoftmaxOutput(data=fc1, name='softmax')


/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
def train():
    
    ## Data Type: cifar-10
    aug_level = 1
    num_classes = 8
    depth = 164
    bn_mom = 0.9
    workspace = 512
    memonger = False
    # depth should be one of 110, 164, 1001,...,which is should fit (args.depth-2)%9 == 0
    if((depth-2)%9 == 0 and depth >= 164):
        per_unit = [(depth-2)/9]
        filter_list = [16, 64, 128, 256]
        bottle_neck = True
    elif((depth-2)%6 == 0 and depth < 164):
        per_unit = [(depth-2)/6]
        filter_list = [16, 16, 32, 64]
        bottle_neck = False
    else:
        raise ValueError("no experiments done on detph {}, you can do it youself".format(depth))
    units = per_unit*3
    units = [int(i) for i in units]
    
    
    symbol = resnet(units=units, num_stage=3, filter_list=filter_list, num_class=num_classes,
                    data_type="cifar10", bottle_neck = bottle_neck, 
                    bn_mom=bn_mom, workspace=workspace,
                    memonger = memonger)
    print(symbol)
    
    ctx = mx.cpu()
    epoch_size = 3
    begin_epoch = 0
    
    train_iter = mx.io.ImageRecordIter(
        path_imgrec = "/home/jiaming/code/github/DeepGlint-Work/Facial/scripts/test.rec", # The target record file.
        path_imgidx = "/home/jiaming/code/github/DeepGlint-Work/Facial/scripts/test.idx",
        data_shape=(3, 224, 224), # Output data shape; 227x227 region will be cropped from the original image.
        batch_size=4, # Number of items per batch.
        resize=256, # Resize the shorter edge to 256 before cropping.
        rand_crop = True,
        rand_mirror = True,
        shuffle = True
    )
    
    test_iter = mx.io.ImageRecordIter(
        path_imgrec = "/home/jiaming/code/github/DeepGlint-Work/Facial/scripts/vali.rec", # The target record file.
        path_imgidx = "/home/jiaming/code/github/DeepGlint-Work/Facial/scripts/vali.idx",
        data_shape=(3, 224, 224), # Output data shape; 227x227 region will be cropped from the original image.
        batch_size=4, # Number of items per batch.
        resize=256, # Resize the shorter edge to 256 before cropping.
        shuffle = False,
        rand_crop = False,
        rand_mirror = False,
    )
    
    model = mx.model.FeedForward(
        ctx                 = mx.cpu(),
        symbol              = symbol,
        arg_params          = None,#arg_params,
        aux_params          = None,#aux_params,
        num_epoch           = 3,
        begin_epoch         = 0,
        learning_rate       = 0.1,
        momentum            = 0.9,#args.mom,
        wd                  = 0.00001,#wd: weight decay
        optimizer           = 'sgd',
        # optimizer          = 'sgd',
        initializer         = mx.init.Xavier(rnd_type='gaussian', factor_type="in", magnitude=2),
        #lr_scheduler        = multi_factor_scheduler(begin_epoch, epoch_size, step=[120, 160], factor=0.1)
        #                     if args.data_type=='cifar10' else
        #                     multi_factor_scheduler(begin_epoch, epoch_size, step=[30, 60, 90], factor=0.1),
        )

    
    model.fit(
        X                  = train_iter,
        eval_data          = test_iter,
        eval_metric        = ['acc', 'ce'], #if args.data_type=='cifar10' else
                             #['acc', mx.metric.create('top_k_accuracy', top_k = 5)],
        #kvstore            = kv,
        batch_end_callback = mx.callback.Speedometer(3,50))#,args.batch_size, args.frequent),
        #epoch_end_callback = checkpoint)

In [ ]:
if __name__ == '__main__':
    train()

<Symbol softmax>


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:69: DeprecationWarning: mxnet.model.FeedForward has been deprecated. Please use mxnet.mod.Module instead.
/home/jiaming/.local/lib/python3.6/site-packages/mxnet/model.py:591: DeprecationWarning: Calling initializer with init(str, NDArray) has been deprecated.please use init(mx.init.InitDesc(...), NDArray) instead.
  self.initializer(k, v)
/home/jiaming/.local/lib/python3.6/site-packages/mxnet/model.py:597: DeprecationWarning: Calling initializer with init(str, NDArray) has been deprecated.please use init(mx.init.InitDesc(...), NDArray) instead.
  self.initializer(k, v)
